In [1]:
#export
import json
import sys
import os
import os.path
from pathlib import Path
from subprocess import call
import sys # system libraries, like arguments (argv)
import re # regular expressions
import pandas as pd


from timeit import default_timer as timer
import time


In [2]:
#Input files
p = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/nextflow/RNAseq-Biome-Nextflow/bin/test/jgi_test/"
f1=p+"test.tsv"
fasta=p+"transcripts.fasta"


f = os.path.basename(f1)
fout = f[:-4]
#NODE_1_length_6153_cov_459.385_g0_i0	gi|18497249|gb|AC104623.4|	98.953	0.0	9606	N/A	N/A	N/A	Homo sapiens BAC clone RP11-542H15 from 2, complete sequence


### Remove human taxid number before jgi call

In [3]:
#export
def removeHuman(f1,fout):
    my_cols=[
    "contig",
    "blast_pident",
    "blast_sseqid",
    "blast_evalue",
    "taxid",
    "blast_sscinames",
    "blast_scomnames",
    "blast_sskingdoms",
    "blast_stitle"
    ]
    df=pd.read_csv(f1, dtype={
                            "contig":str,
                            "blast_pident":str,
                            "blast_sseqid":str,
                            "blast_evalue":str,
                            "taxid":str,
                            "blast_sscinames":str,
                            "blast_scomnames":str,
                            "blast_sskingdoms":str,
                            "blast_stitle":str
                            }, sep='\t', names=my_cols)

    del df["blast_sscinames"]
    del df["blast_scomnames"]
    del df["blast_sskingdoms"]
    df["filename"] = fout
    
    df['contig'] = df.drop_duplicates(subset=['contig'], keep="first")
    df.to_csv(f'{fout}_initial_contigs.txt',index=None,sep="\t")

    
    def getDiff(taxid,df):
        df2 = df[df["taxid"]== taxid]
        df2.to_csv(f'{fout}_{taxid}.txt',index=None,sep="\t")
        df = df[df["taxid"]!= taxid]
        return df
    
    df = getDiff(taxid="9606",df=df) # Remove human
    df = getDiff(taxid="10090",df=df) # Remove mouse
    return df

In [22]:
df = removeHuman(f1,fout)
df.head()

,contig,blast_pident,blast_sseqid,blast_evalue,taxid,blast_stitle,filename
0,NODE_1_length_617_cov_20.523132_g0_i0,gi|694943029|ref|XM_508242.4|,99.514,0.0,9598,PREDICTED: Pan troglodytes hemoglobin subunit ...,test
3,NODE_4_length_507_cov_6.066372_g3_i0,gi|690911020|emb|LN418807.1|,92.669,0.0,99802,Spirometra erinaceieuropaei genome assembly S_...,test
4,NODE_5_length_479_cov_9.966981_g4_i0,gi|1621881971|ref|XR_003713549.1|,100.000,0.0,491861,PREDICTED: Grammomys surdaster 28S ribosomal R...,test
6,NODE_7_length_392_cov_4.225519_g6_i0,gi|1021673813|gb|KX061890.1|,100.000,0.0,9544,"Macaca mulatta precursor 48S ribosomal RNA, 18...",test
8,NODE_10_length_323_cov_2.470149_g9_i0,gi|1622917165|ref|XR_003725683.1|,99.670,5.30e-154,9544,PREDICTED: Macaca mulatta 28S ribosomal RNA (L...,test


### First pass through jgi to remove chordata and viridiplante


In [4]:
#export
def firstJGIQuery(df,fout):
    
    taxids=df['taxid'].unique()
    df_ids = pd.DataFrame(taxids) 
    df_ids = df_ids.rename(columns = {0:"taxid"})
    
    df_ids["jgi_tab"] = "NA"
    df_ids["jgi_json"] = "NA"
    i = 0
    R = len(df_ids)
    for x in range(R):
        print(f'Querying {i+1} of {R} taxids')
        ids = df_ids["taxid"].iloc[x]
        Q1 = "curl https://taxonomy.jgi-psf.org/sc/id/" + str(ids)
        result = os.popen(Q1).read()
        df_ids["jgi_tab"].iloc[x] = result
        
        #Get json return
        
        jasonQ1 = "curl https://taxonomy.jgi-psf.org/id/" + str(ids)
        result = os.popen(jasonQ1).read()
        j = json.loads(result)
        j = j[list(j.keys())[0]]
        j = str(j)
        df_ids["jgi_json"].iloc[x] = j  
        
        i+= 1
        
        

    df = pd.merge(df,df_ids,on="taxid")
    # To remove "Chordata","Viridiplantae",
    # Will remove at jgi json step "synthetic","artificial","PREDICTED"
    df_chordata = df[df["jgi_tab"].str.contains("Chordata")==True]
    df_virid = df[df["jgi_tab"].str.contains("Viridiplantae")==True]

    df_chordata.to_csv(f'{fout}_Chordata.txt',sep="\t",index=None)
    df_virid.to_csv(f'{fout}_Viridiplantae.txt',sep="\t",index=None)
        
    df_artificial = df[df["jgi_tab"].str.contains("synth|vector|Vector|artificial")==True]
    df_artificial.to_csv(f'{fout}_Artificial.txt',sep="\t",index=None)
    
    df = df[df["jgi_tab"].str.contains("Chordata|Viridiplantae")==False]
    df = df[df["jgi_tab"].str.contains("synth|vector|Vector|artificial")==False]
        
    return df


In [24]:
df = firstJGIQuery(df,fout)
df.head()

Querying 1 of 7 taxids
Querying 2 of 7 taxids
Querying 3 of 7 taxids
Querying 4 of 7 taxids
Querying 5 of 7 taxids
Querying 6 of 7 taxids
Querying 7 of 7 taxids


,contig,blast_pident,blast_sseqid,blast_evalue,taxid,blast_stitle,filename,jgi_tab,jgi_json
1,NODE_4_length_507_cov_6.066372_g3_i0,gi|690911020|emb|LN418807.1|,92.669,0.0,99802,Spirometra erinaceieuropaei genome assembly S_...,test,sk:Eukaryota;Opisthokonta;k:Metazoa;Eumetazoa;...,"{'name': 'Spirometra erinaceieuropaei', 'tax_i..."
7,NODE_24_length_242_cov_1.005348_g23_i0,gi|1248686583|gb|CP023560.1|,82.243,3.60e-15,1280,Staphylococcus aureus strain NMR08 chromosome,test,sk:Bacteria;Terrabacteria group;p:Firmicutes;c...,"{'name': 'Staphylococcus aureus', 'tax_id': 12..."



## Match fasta to tab

### Write fasta to tabular output for easy matching

In [5]:
#export
def matchFasta(df,fasta,fout):
    c11 = fout+"_newfasta1.fast"
    c12 = fout+"_newfasta2.fast"
    with open(fasta,'r', newline="\n") as infile, open(c11, 'w') as outfile:
        for line in infile:
            if ">" in line:
                line = line.replace('\n', '\t')
                line = line.replace('>', '\n')
                outfile.write(line)
            else:
                line = line.replace('\n', 'RTN666')
                outfile.write(line)
        infile.close()
        outfile.close()

    with open(c11,'r', newline="\n") as infile, open(c12, 'w') as outfile:
        for line in infile:
            if "NODE" not in line:
                line = line.replace('\n', '')
                line = line.replace('\r', '')
                outfile.write(line)
            else:
                outfile.write(line)
        infile.close()
        outfile.close()

    my_cols=['contig', 'fasta_with_returnsAdded']
    df_fa=pd.read_csv(c12, index_col=['contig'], sep='\t', names=my_cols)
    df = pd.merge(df, df_fa, on='contig') # Dont run this cell more than once
    df["fasta"] = df['fasta_with_returnsAdded'].str.replace("RTN666","")
    df["length"] = df["fasta"].str.len()
    df.to_csv(fout+"_jgi_filtered_df.txt",sep="\t",index=None)
    
    # Write out fasta
    with open(fout+"_jgi_filtered.fasta","w") as outfile:
        for x in range(len(df)):
            nodename = ">"+df["contig"].iloc[x]+"\n"
            fa = df['fasta_with_returnsAdded'].iloc[x]
            fa = fa.replace('RTN666', '\n')
            outfile.write(nodename+fa)
            
    
    return df


In [26]:
df2 = df.copy()
df2 = matchFasta(df=df2,fasta=fasta,fout=fout)
df2.head()

,contig,blast_pident,blast_sseqid,blast_evalue,taxid,blast_stitle,filename,jgi_tab,jgi_json,fasta_with_returnsAdded,fasta,length
0,NODE_4_length_507_cov_6.066372_g3_i0,gi|690911020|emb|LN418807.1|,92.669,0.0,99802,Spirometra erinaceieuropaei genome assembly S_...,test,sk:Eukaryota;Opisthokonta;k:Metazoa;Eumetazoa;...,"{'name': 'Spirometra erinaceieuropaei', 'tax_i...",ATGGAATGGATTGAACCCGAATGGAATGGAAAGGAATGGAATAAAC...,ATGGAATGGATTGAACCCGAATGGAATGGAAAGGAATGGAATAAAC...,507
1,NODE_24_length_242_cov_1.005348_g23_i0,gi|1248686583|gb|CP023560.1|,82.243,3.60e-15,1280,Staphylococcus aureus strain NMR08 chromosome,test,sk:Bacteria;Terrabacteria group;p:Firmicutes;c...,"{'name': 'Staphylococcus aureus', 'tax_id': 12...",GGAGTGGAGTGGAATGGAGTGGAATGGAATGGAATGAGGTGGAATG...,GGAGTGGAGTGGAATGGAGTGGAATGGAATGGAATGAGGTGGAATG...,242


In [6]:
#export
import ast

def toJSON(df,fout):
    df_J = df[["taxid","jgi_json"]]
    del df["jgi_json"]
    
    df.to_json(f"{fout}_temp.json",orient='index')
    with open(f"{fout}_temp.json") as fo:
        contig_d = json.load(fo)
        for c in contig_d.keys():
            d = contig_d[c] 
            taxid = d["taxid"]  
            df_temp = df_J[df_J["taxid"]==taxid]
            j = df_temp["jgi_json"].iloc[0]
            jdic = ast.literal_eval(j)
            d["jgi_json"] = jdic
    
        with open(f"{fout}_jgi.json", "w") as write_file:
            json.dump(contig_d, write_file)
    

In [29]:
df2 = df2.copy()

toJSON(df=df2,fout=fout)

In [30]:

with open(f"{fout}_jgi.json") as fo:
    contig_d = json.load(fo)

contig_d
contig_d["0"]["jgi_json"]["phylum"]["name"]        

'Platyhelminthes'

In [7]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='filter fastas and turn to json')
        parser.add_argument('--file', action= 'store', metavar='file') 
        parser.add_argument('--fasta', action= 'store', metavar='fasta') 
        args = parser.parse_args()
        return args


In [8]:
#export
if __name__=="__main__":
    args = parse_arguments()
    f = args.file
    fasta = args.fasta
    
    #Run all the commands
    fbase = os.path.basename(f)
    fout = fbase[:-4]
    df = removeHuman(f,fout)
    df = firstJGIQuery(df,fout)
    df = matchFasta(df,fasta,fout)
    toJSON(df,fout)
    
    

usage: ipykernel_launcher.py [-h] [--file file] [--fasta fasta]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m/Library/Jupyter/runtime/kernel-1b870906-9f32-45b0-99a6-059b137bf150.json


SystemExit: 2

/Users/m/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
import fire
!python3 notebook2script.py jgiJSON.ipynb

Converted jgiJSON.ipynb to nb_jgiJSON.py
